In [2]:
import json
json_scores = open("embedding_model_scores.json", "r").read()
json_scores = json.loads(json_scores)
print(json_scores)

[[[[9, 0.9999999403953552, True], [12, 0.85952228307724, True], [0, 0.6887946128845215, False], [10, 0.6802195906639099, False], [4, 0.6683498620986938, False], [3, 0.6211584210395813, False], [14, 0.6066176891326904, False], [2, 0.6033022999763489, False], [1, 0.590542733669281, False], [6, 0.5806040167808533, False], [11, 0.5718963146209717, False], [8, 0.5671828389167786, False], [13, 0.5589079856872559, False], [5, 0.5535064935684204, False], [7, 0.5520458817481995, False]]], [[[2, 0.9999999403953552, True], [1, 0.9412275552749634, False], [4, 0.9409853219985962, False], [0, 0.9400177597999573, False], [5, 0.9213075637817383, True], [3, 0.9021545648574829, False]]], [[[72, 1.0, True], [43, 0.978547990322113, True], [25, 0.9755032062530518, True], [37, 0.9714478254318237, False], [61, 0.9706946015357971, False], [75, 0.9690395593643188, False], [95, 0.9688886404037476, False], [41, 0.9630057215690613, False], [15, 0.9607592821121216, False], [50, 0.9600955843925476, False], [20, 0.9

In [3]:
len(json_scores)

263

In [4]:
new_list = []
for image in json_scores:
    for forgery_group in image:
        new_list.append(forgery_group)
print(new_list)
print(len(new_list))


[[[9, 0.9999999403953552, True], [12, 0.85952228307724, True], [0, 0.6887946128845215, False], [10, 0.6802195906639099, False], [4, 0.6683498620986938, False], [3, 0.6211584210395813, False], [14, 0.6066176891326904, False], [2, 0.6033022999763489, False], [1, 0.590542733669281, False], [6, 0.5806040167808533, False], [11, 0.5718963146209717, False], [8, 0.5671828389167786, False], [13, 0.5589079856872559, False], [5, 0.5535064935684204, False], [7, 0.5520458817481995, False]], [[2, 0.9999999403953552, True], [1, 0.9412275552749634, False], [4, 0.9409853219985962, False], [0, 0.9400177597999573, False], [5, 0.9213075637817383, True], [3, 0.9021545648574829, False]], [[72, 1.0, True], [43, 0.978547990322113, True], [25, 0.9755032062530518, True], [37, 0.9714478254318237, False], [61, 0.9706946015357971, False], [75, 0.9690395593643188, False], [95, 0.9688886404037476, False], [41, 0.9630057215690613, False], [15, 0.9607592821121216, False], [50, 0.9600955843925476, False], [20, 0.957514

In [5]:
sorted([(1, 2, True), (2, 3, False), (3, 4, True)], key=lambda x: x[2])

[(2, 3, False), (1, 2, True), (3, 4, True)]

In [23]:
def all_true_first(forgery_group: list[tuple[int, float, bool]]):
    if sum([x[2] for x in forgery_group]) < 2:
        return False
    seen_false = False
    for item in forgery_group:
        if item[2] == False:
            seen_false = True
        else:
            if seen_false:
                return False
    return True

bad_groups = [ x for x in new_list if not all_true_first(x)]
good_groups = [ x for x in new_list if all_true_first(x)]
print(len(bad_groups))


199


In [25]:
new_jsondata = []
for forgery_group in new_list:
    group_data = []
    for item in forgery_group:
        group_data.append({
            'index': item[0],
            'score': item[1],
            'is_forgery': item[2]
        })
    new_jsondata.append(group_data)



In [26]:
with open("new_jsondata.json", "w") as f:
    json.dump(new_jsondata, f)


In [41]:
avg_true_diffs = []
true_to_false_diffs = []
last_true_score = []
first_false_score = []

In [42]:
from itertools import pairwise
for forgery_group in good_groups:
    true_count = 0
    sum_diffs = 0
    for item1, item2 in pairwise(forgery_group):
        if not item2[2]:
            true_to_false_diffs.append(abs(item2[1] - item1[1]))
            first_false_score.append(item2[1])
            last_true_score.append(item1[1])
            break
        true_count += 1
        sum_diffs += abs(item2[1] - item1[1])
    avg_true_diffs.append(sum_diffs / true_count)



In [43]:
import pandas as pd
df = pd.DataFrame(list(zip(avg_true_diffs, true_to_false_diffs, first_false_score, last_true_score)), columns=['avg_true_diff', 'true_to_false_diff', 'first_false_score', 'last_true_score'])


In [44]:
df['ratio'] = df['true_to_false_diff'] / df['avg_true_diff']
df

,avg_true_diff,true_to_false_diff,first_false_score,last_true_score,ratio
0,0.140478,0.170728,0.688795,0.859522,1.215337
1,0.012248,0.004055,0.971448,0.975503,0.331095
2,0.019160,0.039025,0.922654,0.961680,2.036776
3,0.004180,0.020138,0.975683,0.995820,4.817882
4,0.006061,0.036039,0.957899,0.993939,5.945817
...,...,...,...,...,...
159,0.046725,0.028592,0.924683,0.953275,0.611918
160,0.024677,0.000409,0.974915,0.975323,0.016567
161,0.021934,0.022742,0.955324,0.978066,1.036826
162,0.031376,0.044914,0.923711,0.968624,1.431492


In [45]:
df.to_csv("df.csv", index=False)

In [8]:
# load pickle file
import pickle
with open("all_scores.pkl", "rb") as f:
    all_scores = pickle.load(f)



In [41]:
test_index = 2

In [42]:
embeddings, similarity_matrix, forgery_groups = all_scores[test_index]

In [43]:
forgery_groups

[[72, 25, 43], [73, 93, 91]]

In [52]:
from sklearn.cluster import  DBSCAN

# Using raw embeddings (K-Means, DBSCAN, etc.)
from sklearn.cluster import DBSCAN
clusters = DBSCAN(eps=0.3, min_samples=2, metric='cosine').fit_predict(embeddings)
clusters

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [47]:
groups = len(forgery_groups) +1

In [48]:
# awesome
from sklearn.cluster import AgglomerativeClustering

# Using raw embeddings
clusters = AgglomerativeClustering(n_clusters=groups, metric='cosine', linkage='average').fit_predict(embeddings)

# Or with precomputed similarity matrix
distance_matrix = 1 - similarity_matrix
clusters = AgglomerativeClustering(n_clusters=groups, metric='precomputed', linkage='average').fit_predict(distance_matrix)

In [51]:
[(i,x) for i,x in enumerate(clusters) if x !=0]

[(0, 2), (12, 1), (82, 1), (107, 1)]

In [39]:
from sklearn.cluster import SpectralClustering

# With precomputed similarity matrix
clusters = SpectralClustering(n_clusters=2, affinity='precomputed', random_state=42).fit_predict(similarity_matrix)
clusters

array([0, 0, 0, 1, 1, 1], dtype=int32)

In [40]:
from sklearn.cluster import KMeans

clusters = KMeans(n_clusters=2, random_state=42).fit_predict(embeddings)
clusters

array([1, 1, 0, 1, 1, 1], dtype=int32)